##### Расчет показателя max_leas_ovd_pay_dt - дата последней оплаченной просроченной задолженности по лизинговым платежам

In [1]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)

In [2]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/dsnc_ovd_pay_days.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/interm/target_data.pkl"
target_data = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_data))
# target_data.head(1)

127968


условие PBD.PAY_DT_ORIG < PRE_CON.DECISION_DATE - 1 вместо PBD.PAY_DT_ORIG < PRE_CON.DECISION_DATE добавлено с целью совпадения с витриной ЦАЛ, поскольку туда данные заносятся данные с опозданием на день в основном

In [4]:
def read_max_leas_ovd_pay_dt():
    sql_query = """ 
                     SELECT DISTINCT PRE.INN,
                                PRE.DECISION_DATE as DECISION_DATE,
                                PDD.SNAPSHOT_DT  as DAILY_SNAPSHOT_DT,
                                PDD.CONTRACT_KEY as CONTRACT_KEY,
                                PDD.CONTRACT_APP_KEY as CONTRACT_APP_KEY,
                                MAX(FDD.MAX_PAY_DT_ORIG) OVER (PARTITION BY PDD.CONTRACT_KEY, PDD.CONTRACT_APP_KEY, PRE.DECISION_DATE) AS MAX_LEAS_OVD_PAY_DT,
                                sysdate as 	INSERT_DT
                FROM (SELECT DISTINCT SNAPSHOT_DT, CONTRACT_KEY, CONTRACT_APP_KEY, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE
                            FROM DM.DM_DETAILS_DAILY
                            WHERE PAYMENT_ITEM_KEY in (9, 1000)
                    ) PDD
                INNER JOIN
                ( SELECT * FROM PRE_CONTRACTS
                ) PRE
                ON PDD.CONTRACT_KEY      = PRE.CONTRACT_KEY
                AND PDD.PLAN_PAY_DT_ORIG < PRE.DECISION_DATE - 1
                LEFT JOIN (SELECT DISTINCT CONTRACT_KEY, CONTRACT_APP_KEY, DECISION_DATE, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT ,
                            MAX(PAY_DT_ORIG) OVER (PARTITION BY CONTRACT_KEY, CONTRACT_APP_KEY, DECISION_DATE,  CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE)         AS MAX_PAY_DT_ORIG ,
                            SUM(NVL(FACT_PAY_AMT,0)) OVER (PARTITION BY CONTRACT_KEY, CONTRACT_APP_KEY, DECISION_DATE, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE) AS FACT_PAY_AMT ,
                            PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE
                          FROM 
                            (SELECT PBD.CONTRACT_KEY, PBD.CONTRACT_APP_KEY, DECISION_DATE, CBC_DESC, PAYMENT_NUM, PLAN_PAY_DT_ORIG, PLAN_AMT , PAY_DT_ORIG,  FACT_PAY_AMT, PAYMENT_ITEM_KEY, CURRENCY_KEY, OFF_SCHEDULE
                            FROM DM.DM_DETAILS_DAILY PBD
                          INNER JOIN
                            ( SELECT * FROM PRE_CONTRACTS 
                            ) PRE_CON
                          ON PBD.CONTRACT_KEY = PRE_CON.CONTRACT_KEY
                          AND PBD.PAY_DT_ORIG < PRE_CON.DECISION_DATE - 1
                          WHERE PAYMENT_ITEM_KEY  IN (9, 1000)
                            AND NVL(FACT_PAY_AMT,0) >= 0
                            AND NVL(PLAN_AMT, 0)     > 0
                            AND PRE_PAY              > 0
                            )) FDD 
                                ON   PDD.CONTRACT_KEY = FDD.CONTRACT_KEY
                                AND PDD.CONTRACT_APP_KEY = FDD.CONTRACT_APP_KEY
                                AND PDD.CBC_DESC = FDD.CBC_DESC
                                AND PDD.PAYMENT_NUM = FDD.PAYMENT_NUM
                                AND PDD.PLAN_PAY_DT_ORIG = FDD.PLAN_PAY_DT_ORIG
                                AND PDD.PLAN_AMT = FDD.PLAN_AMT
                                AND PDD.PAYMENT_ITEM_KEY = FDD.PAYMENT_ITEM_KEY
                                AND PDD.CURRENCY_KEY = FDD.CURRENCY_KEY
                                AND PDD.OFF_SCHEDULE = FDD.OFF_SCHEDULE
                                AND PRE.DECISION_DATE = FDD.DECISION_DATE
                  WHERE (PDD.PLAN_AMT - FDD.FACT_PAY_AMT) <= 0 AND FDD.MAX_PAY_DT_ORIG > PDD.PLAN_PAY_DT_ORIG
                """
                
    df = read_from_oracle(sql_query, title='leas_ovd_pay_dt')
    
    return df

leas_ovd_pay_dt = read_max_leas_ovd_pay_dt()
# leas_ovd_pay_dt.head(1)

Размер считанных данных leas_ovd_pay_dt - (101634, 7)


In [5]:
leas_ovd_pay_dt.rename(columns={'MAX_LEAS_OVD_PAY_DT':'max_leas_ovd_pay_dt', 'DECISION_DATE':'decision_date'}, inplace=True)
leas_ovd_pay_dt['CONTRACT_KEY'] = leas_ovd_pay_dt['CONTRACT_KEY'].astype(str)

In [6]:
leas_ovd_pay_dt.dtypes

INN                            object
decision_date          datetime64[ns]
DAILY_SNAPSHOT_DT      datetime64[ns]
CONTRACT_KEY                   object
CONTRACT_APP_KEY                int64
max_leas_ovd_pay_dt    datetime64[ns]
INSERT_DT              datetime64[ns]
dtype: object

In [7]:
l = len(leas_ovd_pay_dt[leas_ovd_pay_dt['max_leas_ovd_pay_dt'] > leas_ovd_pay_dt['decision_date']])

assert l == 0

In [8]:
leas_ovd_pay_dt.to_pickle(pre+"data/interm/leas_ovd_pay_dt.pkl", compression={'method': 'zip'})
# leas_ovd_pay_dt.head(2)

In [9]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 